In [54]:
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 16.1 Char-RNN
## 16.1.1 훈련 데이터셋 만들기

In [55]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


In [56]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [57]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [58]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [59]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [60]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [61]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [62]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [63]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [64]:
n_steps = 100
window_length = n_steps+1
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [65]:
dataset = dataset.flat_map(lambda window : window.batch(window_length))

In [66]:
dataset

<FlatMapDataset shapes: (None,), types: tf.int32>

In [67]:

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [68]:
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [69]:
dataset = dataset.prefetch(1)

In [70]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## 16.1.4 Char-RNN 모델 만들고 훈련하기

In [71]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id], dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True, dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))    
# ])

# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, steps_per_epoch=train_size//batch_size, epochs=2)

In [72]:
model = keras.models.load_model("stateless_rnn.h5")

## 16.1.5 Char-RNN 모델 사용하기
- 다음글자를 예측하는 모델에 새로운 텍스트를 주입하려면 전처리를 해야함

In [73]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X,max_id)

In [74]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


'u'

In [75]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]],
      dtype=int64)

In [96]:
def next_char(text, temperature,model):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [97]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1,model=model)

'u'

In [101]:
def complete_text(text, model, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature,model=model)
    return text

In [79]:
tf.random.set_seed(42)
print(complete_text("t", temperature=0.2))

the store--

menenius:
why, sir, you must not stay 


In [80]:
print(complete_text("t",temperature=1))

toke that first enough to speak
at soed as dures at


In [81]:
print(complete_text("t",temperature=2))

tpeniomevily
weel?gmaze you bevile deficurs:
-theem


## 16.1.7 Stateful RNN

In [82]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window : window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows : (windows[:,:-1], windows[:,1:]))
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth=max_id),Y_batch))
dataset = dataset.prefetch(1)

In [83]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window : window.batch(window_length))
    datasets.append(dataset)

dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows : tf.stack(windows))
dataset = dataset.repeat().map(lambda windows : (windows[:,:-1], windows[:,1:]))
dataset = dataset.map(lambda X_batch, Y_batch : (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [84]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [85]:
class ResetStatesCallback(keras.callbacks.Callback): # 에포크 끝마다 텍스트를 다시 시작하기 전에 상태를 재설정
    def on_batch_begin(self, epochs, logs):
        self.model.reset_states()

In [92]:
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# steps_per_epoch = train_size // batch_size // n_steps
# history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50, callbacks=[ResetStatesCallback()])

# model.save("stateful_rnn.h5")

In [93]:
model = keras.models.load_model("stateful_rnn.h5")

# 16.2 Sentiment Analysis